# Aggregeren en pivoteren
## vertrek van bi_cleaned

In [ ]:
from pathlib import Path
import requests
BI_CSV = 'bi_cleaned.csv'
bi_cleaned_path = Path(BI_CSV)
if not bi_cleaned_path.exists():
    data = requests.get('https://learn.walsoftcomputers.com/csv/cleaned_bi.csv')
    with open(BI_CSV, mode='wb') as f:
        f.write(data.content)
else:
    print('bestand bestaat al.')
with open(BI_CSV, 'r') as f:
    for _ in range(5):
        print(f.readline(), end='')

## inlezen in pandas
Let op met de encoding (windows-1252)

In [ ]:
import pandas as pd
BI_CSV = 'bi_cleaned.csv'
df = pd.read_csv(BI_CSV, encoding='windows-1252', delimiter=',')
print("aantal rijen:", df.shape[0])
df.info()
df.describe()

In [ ]:
df.head()

## De gemiddelde punten voor Python en DB
Hiervoor kunnen we natuurlijk de mean()-functie gebruiken

In [ ]:
df[['Python', 'DB']].mean()

## In welk land wonen de beste Python programmeurs?
Hiervoor moeten we groeperen op country en vervolgens het gemiddelde berekenen

In [ ]:
df.groupby(by='country')[['Python']].mean().sort_values('Python', ascending=False)

## Maar hebben we wel genoeg data om gemiddeldes te vergelijken?
Een gemiddelde van veel studenten is betrouwbaarder dan een gemiddelde van weinig studenten. De functie .value_counts() hebben we al gezien. Maar we kunnen ook de count()-functie van groupby gebruiken

In [ ]:
display(df.value_counts('country'))
df.groupby('country')['Python'].count()

## Meerdere groeperingskeys
We kunnen groeperen op prevEducation en gender

In [ ]:
df.groupby(['prevEducation', 'gender'])['Python'].mean().index

# De unstack()-functie
Het resultaat van de vorige groupbe is een Series met een multiIndex (een index van twee niveaus: prevEducation en gender). Een Series met een multiIndex kan omgezet worden naar een DataFrame waarbij 1 onderdeel van de multiIndex de kolomreeks wordt

In [ ]:
df.groupby(['prevEducation', 'gender'])['Python'].mean().unstack()

## De pivot_table() functie
We kunnen hetzelfde resultaat verkrijgen met de pivot_table() functie:

In [ ]:
df.pivot_table('Python', index='prevEducation', columns='gender', aggfunc='mean')

## Een pivottable met leeftijdbins
Met pd.cut kunnen we bins maken. Het argument observed=True zorgt ervoor dat lege categorieën niet getoond worden. Maar aangezien we geen categoriekolom hebben, maakt dat geen verschil (maar het voorkomt een warning.)

In [ ]:
leeftijd = pd.cut(df['Age'], [0, 25, 60, 100])
df.pivot_table('Python', index=['prevEducation', leeftijd], columns='gender', aggfunc='mean', observed=True )